In [ ]:
#@TODO check which of the following is relevant
#@TODO: get _country = provience maybe? relevant
#@TODO: get _city relevant 
#@TODO: get _poweer_plant relevant
#@TODO: get invistor, transaction and debt 

# Imports & Global Variables

In [5]:
import pandas as pd
import numpy as np

# import custom-made functions
from functions.cleaning import get_variables, create_placeholder, create_unique_id, get_variables_advanced, preprocess_text
from functions.cleaning import  check_if_new_country_names, update_country_names
from functions.joining import join_pp_until_country, join_equity_transaction, join_debt_transaction
from functions.final_tables_creation import get_tables_to_create_empty

# other useful libraries
import itertools
from statistics import mean, stdev
import statistics

# for the matching of power plant names
from thefuzz import process
from thefuzz import fuzz

from functions.joining import join_pp_full_debt_full_equity_full
from functions.cleaning import my_read_excel, my_reset_index
from functions.cleaning import get_thresholds_methods_dicts
from functions.matching import find_matches_equity_and_debt_complete
from functions.matching import find_matches_power_plant_or_fill_PP_key
from functions.cleaning import extract_between_outside_underscores
from functions.cleaning import print_final_statistics, fix_investor_names, clean_primary_fuel_column
from functions.cleaning import count_sources_in_id
from functions.matching import prepare_pp_full_for_matching_commissioning_year
from functions.cleaning import fix_duplicated_power_plants_keys

In [6]:
# needed folder paths
input_folder = "Cleaned Data/" # or we can put this info in the "files name" sheet
intermediate_folder = "Intermediate Results/" # to save some intermediate results
current_final_tables_folder = "Current Final Tables/" 


In [7]:
# load the excel containing all the mapping information
dictionary_fuel_name = "Dictionaries.xlsx"
dictionary_fuel = pd.read_excel(dictionary_fuel_name, sheet_name=None) # read all the sheets that we will be using togehter
print(len(dictionary_fuel))

13


In [8]:
#@TODO thresholds and methods goes here

In [9]:
# the tables we are creating here
tables_to_create = ["Power Plant", "City", "Country", "Debt", "Equity", "Transaction", "Investor"]
investments_tables = ["Equity", "Transaction", "Investor"]

In [10]:
# already existing final tables
debt_file = "Debt.csv"
power_plant_file = "Power Plant.csv"
city_file = "City.csv"
country_file = "Country.csv"
bridge_file = "CITYKEY_BRIDGE_PP_C.csv"
equity_file = "Equity.csv"
transaction_file = "Transaction.csv"

In [11]:
source_name = "AidData"

# names of the final tables to be used here
equity_name = "Equity"
debt_name = "Debt"

# saving information
saving_suffix = "_AidData.csv"

# 1. Read Data

In [12]:
aid = pd.read_excel("/Users/mariam/Vs code projects/WRi-COFI-streamlit/cleaning/AidData - New Plants with extracted info.xlsx")

In [13]:
aid_full =aid.copy(deep=True)

In [14]:
aid.head()

,AidData Record ID,Title,Recipient ISO-3,Commitment Year,Funding Agencies,Co-financing Agencies,Amount (Nominal USD),Amount (Constant USD 2021),Adjusted Amount (Constant USD 2021),Flow Type,M&A,Description,to_keep,reason,primary_fuel,installed_capacity,additional_info,pattern name
0,96176,"CCB, BoC, China CITIC Bank, and China Bohai Ba...",PAK,2021,Bank of China (BOC)|China Bohai Bank|China CIT...,Credit Suisse|Deutsche Bank|Standard Chartered...,1.664000e+08,1.664000e+08,1.664000e+08,Loan,NaN,"On September 13, 2021, Bin Qasim 3 CCGT — a sp...",True,Power Station,NaN,900.0,Bin Qasim,second pattern
1,96035,ICBC participates in PHP 1.6 billion syndicate...,PHL,2021,Industrial and Commercial Bank of China (ICBC),Security Bank Cooperation,1.623706e+07,1.623706e+07,1.623706e+07,Loan,NaN,"On January 20, 2021, the Philippines’ Departme...",True,new pattern,solar pv,NaN,Phases 3A and 3B of the Bataan,fuels pattern
2,92685,Africa Growing Together Fund provides $20 mill...,TZA,2021,People's Bank of China (PBC),Africa Development Bank,2.000000e+07,2.000000e+07,2.000000e+07,Loan,NaN,"On May 22, 2014, the African Development Bank ...",True,new pattern,hydro,44.8,Malagarasi,first pattern
3,92631,Bank of China contributes to $750 million synd...,UZB,2021,Bank of China (BOC),Asian Infrastructure Investment Bank (AIIB)|Eu...,3.000000e+08,3.000000e+08,3.000000e+08,Loan,NaN,"In 2019, the Government of Uzbekistan initiate...",True,Power Project,NaN,1500.0,Sirdarya Independent,second pattern
4,96465,Bank of China contributes to $81 million syndi...,VNM,2021,Bank of China (BOC),Asian Development Bank|Export Finance Australi...,4.050000e+07,4.050000e+07,4.050000e+07,Loan,NaN,"In August 2021, Lien Lap Wind Power Joint Stoc...",True,Power Project,onshore wind|offshore wind,NaN,Lotus,fuels pattern


In [15]:
aid.shape

(197, 18)

## 1.2 Load data for existing Power Plants and Debt data for matching purposes

In [16]:
columns_to_keep_pp = ["PP_key", "power_plant_name", "primary_fuel", "province", "country"]
columns_to_keep_equity = ["equity_id", "PP_key", "equity_investment_type",  "equity_investment_year", "investor_name", "amount"]
columns_to_keep_debt = ["debt_id", "PP_key", "debt_investment_year", "investor_name", "amount"]

In [17]:
pp_full, db_pp,  eq_pp = join_pp_full_debt_full_equity_full(columns_to_keep_pp, columns_to_keep_debt, columns_to_keep_equity, "Current Final Tables/", get_commissioning_year=True)

Retriving data from folder "Current Final Tables/"....
Data was last modified on: 2024-11-21
Joining Power Plant and briding table, all power plants preserved: False
Joining with city, all power plants preserved: False
Joining with country, all power plants preserved: True


Retriving data from folder "Current Final Tables/"....
Data was last modified on: 2024-11-21
Joining Debt and Transaction, all Debt entries had matching transaction: True
Joining Debt full and Power Plant full, all Debt entries are preserved: False


Retriving data from folder "Current Final Tables/"....
Data was last modified on: 2024-11-21
Joining Equity and Transaction, all Equity entries had matching transaction: True
Joining Equity full and Power Plant full, all Equity entries are preserved: False


In [19]:
pp_full.head()

,PP_key,power_plant_name,primary_fuel,province,country,commissioning_year,year_range
0,PLANTKEY_FDIMARKETS_0,NaN,solar,unnamed province 0,serbia,NaN,NaN
1,PLANTKEY_FDIMARKETS_1,NaN,hydropower,tashkent viloyati,uzbekistan,NaN,NaN
2,PLANTKEY_FDIMARKETS_1,NaN,hydropower,tashkent viloyati,uzbekistan,NaN,NaN
3,PLANTKEY_FDIMARKETS_1,NaN,hydropower,tashkent viloyati,uzbekistan,NaN,NaN
4,PLANTKEY_FDIMARKETS_1,NaN,hydropower,tashkent viloyati,uzbekistan,NaN,NaN


In [20]:
db_pp.head()

,debt_id,PP_key,debt_investment_year,investor_name,amount,power_plant_name,primary_fuel,province,country,commissioning_year,year_range
0,DEBTID_SAISCLA_IADGEGI_17,PLANTKEY_WEPPGPPD_6412,2012.0,china energy engineering corporation ceec,21.773,MABUBAS,hydropower,bengo,angola,2012.0,2012.0-2012.0
1,DEBTID_SAISCLA_IADGEGI_17,PLANTKEY_WEPPGPPD_6412,2012.0,export import bank of china,1003.000,MABUBAS,hydropower,bengo,angola,2012.0,2012.0-2012.0
2,DEBTID_SAISCLA_IADGEGI_18,PLANTKEY_WEPPGPPD_11099,2010.0,industrial and commercial bank of china,60.000,LOMAUM,hydropower,benguela,angola,NaN,missing_year_wepp_202336-missing_year_wepp_202336
3,DEBTID_SAISCLA_IADGEGI_18,PLANTKEY_WEPPGPPD_11099,2010.0,export import bank of china,15.000,LOMAUM,hydropower,benguela,angola,NaN,missing_year_wepp_202336-missing_year_wepp_202336
4,DEBTID_SAISCLA_IADGEGI_20,PLANTKEY_WEPPGPPD_90576,2015.0,industrial and commercial bank of china,837.500,SOYO-I,gas,zaire,angola,2017.0,2017.0-2019.0


In [21]:
eq_pp.head()

,equity_id,PP_key,equity_investment_type,equity_investment_year,investor_name,amount,power_plant_name,primary_fuel,province,country,commissioning_year,year_range
0,EQUITYID_BUCGP_0,PLANTKEY_BUCGP_EQ_0,greenfield,NaN,power construction corporation of china,NaN,Pulau Baai Bengkulu Coal Plant,coal,bengkulu,indonesia,2021.0,2021-2021
1,EQUITYID_BUCGP_1,PLANTKEY_BUCGP_EQ_1,m&a,NaN,china datang corporation ltd,NaN,Tangerang Gas Plant,gas,banten,indonesia,2020.0,2020-2020
2,EQUITYID_BUCGP_2,PLANTKEY_BUCGP_EQ_2,m&a,NaN,china general nuclear power corp,NaN,Jimah Coal Plant,coal,negeri sembilan,malaysia,2016.0,2016-2016
3,EQUITYID_BUCGP_3,PLANTKEY_BUCGP_EQ_3,m&a,NaN,china general nuclear power corp,NaN,Meghnaghat Pendekar Gas Plant,gas,dhaka,bangladesh,2016.0,2016-2016
4,EQUITYID_BUCGP_4,PLANTKEY_BUCGP_EQ_4,m&a,NaN,state power investment corp ltd,NaN,Sao Simao Paranaiba Hydropower Plant,hydropower,goiás,brazil,2018.0,2018-2018


In [22]:
count_sources_in_id(pp_full, "PP_key", "PP full")

Rows in PP full from BUCGEF: 102
Rows in PP full from BUCGP_DT: 11
Rows in PP full from BUCGP_EQ: 312
Rows in PP full from FDIMARKETS: 438
Rows in PP full from IJGLOBAL_DT: 95
Rows in PP full from IJGLOBAL_EQ: 92
Rows in PP full from REFINITIVLOAN: 971
Rows in PP full from REFINITIVMA: 184
Rows in PP full from WEPPGPPD: 184423


In [23]:
count_sources_in_id(db_pp, "debt_id")

Rows in data from BUCGEF: 902
Rows in data from BUCGP: 113
Rows in data from IJGLOBAL: 893
Rows in data from REFINITIVLOAN: 1199
Rows in data from SAISCLA_IADGEGI: 25


In [24]:
count_sources_in_id(eq_pp, "equity_id", )

Rows in data from BUCGP: 587
Rows in data from FDIMARKETS: 4260
Rows in data from IJGLOBAL: 811
Rows in data from REFINITIVMA: 205


## 1.3 Load Debt and Equity Tables

In [25]:
equity = my_read_excel(current_final_tables_folder, equity_file)
equity.head()

Equity.csv: last modified on: 2024-11-21


,equity_id,PP_key,total_investment_amount,equity_investment_type,equity_investment_year,bu_id_bu_cgp,bu_id,fdi_id,rma_id,j_id
0,EQUITYID_BUCGP_0,PLANTKEY_BUCGP_EQ_0,0.0,greenfield,NaN,ID.F.027.2,NaN,NaN,NaN,NaN
1,EQUITYID_BUCGP_1,PLANTKEY_BUCGP_EQ_1,0.0,m&a,NaN,ID.F.026.1,NaN,NaN,NaN,NaN
2,EQUITYID_BUCGP_2,PLANTKEY_BUCGP_EQ_2,0.0,m&a,NaN,MY.F.002.1; MY.F.002.2,NaN,NaN,NaN,NaN
3,EQUITYID_BUCGP_3,PLANTKEY_BUCGP_EQ_3,0.0,m&a,NaN,BD.F.002.1; BD.F.002.2; BD.F.002.3,NaN,NaN,NaN,NaN
4,EQUITYID_BUCGP_4,PLANTKEY_BUCGP_EQ_4,0.0,m&a,NaN,BR.F.110.1; BR.F.110.2; BR.F.110.3; BR.F.110.4...,NaN,NaN,NaN,NaN


In [26]:
debt = my_read_excel(current_final_tables_folder, debt_file)
debt.head()

Debt.csv: last modified on: 2024-11-21


,debt_id,PP_key,debt_investment_year,debt_investment_amount,investment_cofinanced_bu,number_of_lenders,bu_id_bu_cgp,bu_id,j_id,r_id
0,DEBTID_SAISCLA_IADGEGI_0,PLANTKEY_SAISCLA_IADGEGI_0,2013.0,458.3924,NaN,NaN,NaN,AFR.026,NaN,NaN
1,DEBTID_SAISCLA_IADGEGI_1,PLANTKEY_SAISCLA_IADGEGI_1,2005.0,13.1348,NaN,NaN,NaN,AO.003.19,NaN,NaN
2,DEBTID_SAISCLA_IADGEGI_2,PLANTKEY_SAISCLA_IADGEGI_2,2005.0,25.4238,NaN,NaN,NaN,AO.003.20,NaN,NaN
3,DEBTID_SAISCLA_IADGEGI_3,PLANTKEY_SAISCLA_IADGEGI_3,2004.0,40.0584,NaN,NaN,NaN,AO.003.21,NaN,NaN
4,DEBTID_SAISCLA_IADGEGI_4,PLANTKEY_SAISCLA_IADGEGI_4,2005.0,34.5309,NaN,NaN,NaN,AO.003.22,NaN,NaN


# 2. Clean Data

## 2.1 Vertical Slicing

In [ ]:
aid_variables = [] #@TODO add variables here

### Country

### Primary Fuel

### investment_type

### Rows that don't have equity nor debt data

### investor_name

# 3. Match with existing data

# 4. Find matches with Power Plant

# 5. Create Tables